In [5]:
import torch

import torch.nn as nn

import numpy as np

In [6]:
# Step 1: Tokenize corpus into words

corpus = "deep learning is fun and deep learning is powerful"

words = corpus.split()

In [7]:
# Step 2: Build vocabulary

vocab = sorted(set(words))

vocab_size = len(vocab)


word_to_idx = {word: i for i, word in enumerate(vocab)}

idx_to_word = {i: word for word, i in word_to_idx.items()}

print("Vocabulary:", vocab)

print("word_to_idx:", word_to_idx)

Vocabulary: ['and', 'deep', 'fun', 'is', 'learning', 'powerful']
word_to_idx: {'and': 0, 'deep': 1, 'fun': 2, 'is': 3, 'learning': 4, 'powerful': 5}


Model Architecture

In [8]:
class WordRNN(nn.Module):

    def __init__(self,vocab_size, embed_dim, hidden_dim):
        super(WordRNN, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embed_dim)

        self.rnn = nn.RNN(embed_dim, hidden_dim, batch_first=True)

        self.fc = nn. Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden):

        embedded = self.embedding(x)

        out, hidden = self.rnn(embedded, hidden)

        out = self.fc(out)

        return out, hidden

nn.Embedding maps words to dense vectors

RNN processes word-by-word

Final output is fed into a linear layer to predict the next word in the sequence

🔧 Define Model + Hyperparams

In [9]:
# Parameters

embed_dim = 10

hidden_dim  = 20

model = WordRNN(vocab_size, embed_dim, hidden_dim)